In [18]:
import numpy as np
import numpy.linalg as la
from matplotlib import pyplot as plt
import pandas as pd
import cmath

# Problem 1-1)

In [19]:
preamble = np.array([1,0,0,1,1,0,1,0,1,1,0,0,0,1,0,0,0,1,1,0,1,0,0,1,1,1,0,0,1,0,1,1])

In [20]:
rx_signal = np.load('rx_signal.npy')

In [21]:
def two_bit_mod(bits):
    assert bits.shape[0] == 2
    if np.equal(bits, np.array([0,0])).all():
        return 1
    elif np.equal(bits, np.array([0,1])).all():
        return 1j
    elif np.equal(bits, np.array([1,0])).all():
        return -1j
    elif np.equal(bits, np.array([1,1])).all():
        return -1
    return 0

def two_bit_modulation(preamble):
    assert preamble.shape[0] % 2 == 0
    modulated_preamble = np.empty(int(preamble.shape[0] / 2), dtype=complex)

    for i in range(0, len(preamble), 2):
        modulated_preamble[int(i / 2)] = two_bit_mod(preamble[i:i+2])
    return modulated_preamble

two_bit_modulation(preamble)

array([-0.-1.j,  0.+1.j, -0.-1.j, -0.-1.j, -1.+0.j,  1.+0.j,  0.+1.j,
        1.+0.j,  0.+1.j, -0.-1.j, -0.-1.j,  0.+1.j, -1.+0.j,  1.+0.j,
       -0.-1.j, -1.+0.j])

## Problem 1-1) i.

In [22]:
# problem 1-i
def problem_1_1():
    preamble = np.array([1,0,0,1,1,0,1,0,1,1,0,0,0,1,0,0,0,1,1,0,1,0,0,1,1,1,0,0,1,0,1,1])
    answer = np.array([-1j, 1j, -1j, -1j, -1, 1, 1j, 1, 1j, -1j, -1j, 1j, -1, 1, -1j, -1]) # Used to check against actual answer
    output = two_bit_modulation(preamble)
    assert np.equal(answer, output).all()
    return output

problem_1_1()

array([-0.-1.j,  0.+1.j, -0.-1.j, -0.-1.j, -1.+0.j,  1.+0.j,  0.+1.j,
        1.+0.j,  0.+1.j, -0.-1.j, -0.-1.j,  0.+1.j, -1.+0.j,  1.+0.j,
       -0.-1.j, -1.+0.j])

## Problem 1-1) ii.

In [23]:
# problem 1-ii
def problem_1_2():
    preamble = np.array([1,0,0,1,1,0,1,0,1,1,0,0,0,1,0,0,0,1,1,0,1,0,0,1,1,1,0,0,1,0,1,1])
    x_hat = two_bit_modulation(preamble)
    y_hat = rx_signal[0:16]
    h = np.average(y_hat / x_hat)
    return h

problem_1_2()

(0.50114281148847+0.3211975378330696j)

## Problem 1-1) iii.

In [24]:
# problem 1-iii
def normalize_range(arr):
    while ((arr < 0).any()):
        arr[(arr < 0)] += 2*np.pi
    while ((arr < 0).any()):
        arr[arr >= 2*np.pi] -= 2*np.pi
    return arr

def two_bit_demodulation(msg):
    possible_bits = np.array([0.0, 0.5*np.pi, np.pi, 1.5*np.pi, 2*np.pi])
    matching_bits = np.array([[0,0],[0,1],[1,1],[1,0],[0,0]])
    return matching_bits[np.argmin(abs(msg - possible_bits))]

def stupid_two_bit_demodulation(msg):
    pass
    
def problem_1_3():
    h = problem_1_2()
    msg = rx_signal[16:]
    original_msg = msg / h
    phase = np.vectorize(cmath.phase)(original_msg)
    phase = normalize_range(phase)
    
    bit_msg = np.empty(len(msg)*2, dtype=int)
    for i in range(0, len(msg)*2, 2):
        two_bit_msg = two_bit_demodulation(phase[int(i/2)])
        bit_msg[i:i+2] = two_bit_msg

    return bit_msg

problem_1_3()

array([0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0])

In [25]:
# Convert bit-array to ascii value
def get_ascii(message_bits):
    me = np.packbits(message_bits)
    return [chr(m) for m in me]
''.join(get_ascii(problem_1_3()))

'Welcome to CS498'

# Problem 1-2)

In [26]:
rx_mimo_0 = np.load('rx_mimo_0.npy')
rx_mimo_1 = np.load('rx_mimo_1.npy')

## Problem 1-2) i.

$y_1 = h_{11}x_1+h_{21}x_2+n_1$

$y_2 = h_{12}x_1+h_{22}x_2+n_2$

$y = Hx + n$

$H^{-1}y = x + H^{-1}n$

In [27]:
def problem_2_1():
    x_hat = two_bit_modulation(preamble)
    y_hat_11 = rx_mimo_0[:16]
    y_hat_21 = rx_mimo_1[:16]
    y_hat_12 = rx_mimo_0[16:32]
    y_hat_22 = rx_mimo_1[16:32]
    h_11 =  np.average(y_hat_11 / x_hat)
    h_12 =  np.average(y_hat_12 / x_hat)
    h_21 =  np.average(y_hat_21 / x_hat)
    h_22 =  np.average(y_hat_22 / x_hat)
    return np.array([[h_11, h_12],[h_21, h_22]])
    
problem_2_1()

array([[ 0.29923584+0.5001769j ,  0.4992434 +0.30066846j],
       [ 0.09917548-0.49943232j, -0.70022036+0.20104199j]])

In [28]:
def problem_2_2():
    H = problem_2_1()
    msg_0 = rx_mimo_0[32:]
    msg_1 = rx_mimo_1[32:]
    original_msg = la.inv(H) @ [msg_0, msg_1]
    phase = np.vectorize(cmath.phase)(original_msg)
    phase = normalize_range(phase)

    decoded_msgs = []
    for m in range(phase.shape[0]):
        bit_msg = np.empty(phase.shape[1]*2, dtype=int)
        for b in range(0, phase.shape[1]*2, 2):
            two_bit_msg = two_bit_demodulation(phase[m][int(b/2)])
            bit_msg[b:b+2] = two_bit_msg
        decoded_msgs.append(bit_msg)
    return decoded_msgs

problem_2_2()

[array([0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0,
        0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
        0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1,
        1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0,
        0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0,
        1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]),
 array([0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0,
        1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
        1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
        1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0])]

## Problem 1-2) iv.

In [29]:
def problem_2_4():
    decoded_msgs = problem_2_2()
    # for i in range(len(decoded_msgs)):
    #     decoded_msgs[i] = get_ascii(decoded_msgs[i])
    return [''.join(get_ascii(msg)) for msg in decoded_msgs]
problem_2_4()

['Hey, does this  ', 'make sense now? ']

# Problem 1-3)

In [30]:
H = np.array([[(1+1j)/np.sqrt(2), (-3+3j)/np.sqrt(2)],[2j, -6]])
H

array([[ 0.70710678+0.70710678j, -2.12132034+2.12132034j],
       [ 0.        +2.j        , -6.        +0.j        ]])

In [174]:
# Check Empirically
# H = problem_2_1()
# msg_0 = rx_mimo_0[32:]
# msg_1 = rx_mimo_1[32:]
# original_msg = la.inv(H) @ [msg_0, msg_1]
# phase = np.vectorize(cmath.phase)(original_msg)
# phase = normalize_range(phase)

# decoded_msgs = []
# for m in range(phase.shape[0]):
#     bit_msg = np.empty(phase.shape[1]*2, dtype=int)
#     for b in range(0, phase.shape[1]*2, 2):
#         two_bit_msg = two_bit_demodulation(phase[m][int(b/2)])
#         bit_msg[b:b+2] = two_bit_msg
#     decoded_msgs.append(bit_msg)

# Encode msg
x_1 = (np.random.rand(16) >= 0.5).astype(int) # np.array([0,1,0,1,0,1])
x_2 = (np.random.rand(16) >= 0.5).astype(int) # np.array([1,1,1,1,1,1])
x = np.array([x_1, x_2])
y = H @ x

# Try decoding
y1, y2 = y
h11, h12 = H[0]
h21, h22 = H[1]

In [175]:
x_ = la.lstsq(H, y, rcond=None)[0]
# x1 = x_[0] / la.norm(x_[0], np.inf)
# x2 = x_[1] / la.norm(x_[1], np.inf)
# x1 = (x_[0] / la.norm(x_[0], np.inf)).astype(int)
# x2 = (x_[1] / la.norm(x_[1], np.inf)).astype(int)
x1 = (x_[0].real / la.norm(x_[0].real, np.inf)).astype(int)
x2 = (x_[0].imag / la.norm(x_[0].imag, np.inf)).astype(int)

In [176]:
# x1_ = x[0][0].imag
# x1 = (x1_ / la.norm(x1_, np.inf)).astype(int)
# x2_ = x[0][1].real
# x2 = (x2_ / la.norm(x2_, np.inf)).astype(int)
# print(x1, x2)

In [181]:
x1, x_1, (x1 == x_1).all()

(array([0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1]),
 array([0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1]),
 True)

In [187]:
x2, x_2, (x2 == x_2).all()

(array([1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0]),
 array([1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0]),
 True)

In [198]:
H1 = H[:,0] / la.norm(H[:,0])
H2 = H[:,1] / la.norm(H[:,1])
H1, H2

(array([0.31622777+0.31622777j, 0.        +0.89442719j]),
 array([-0.31622777+0.31622777j, -0.89442719+0.j        ]))

**Answer:** As you can see, we are able to extract both message from the real and imaginary parts of our least squares answer. This is because in our encoding for our 2nd message, it encodes it into the imaginary dimension, so we are basically using the imaginary part as the encoding for the 2nd message, and the real part as the encoding for the first message. You can also think of this as choosing some $\alpha$ value that makes it so $x_1$ and $x_2$ become orthogonal (in order to decode), and so we can choose $3j$ as that performs a $\pi/2$ rotation allowing us to actually have two orthogonal vectors (note they are not orthonormal).